In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from dicttoxml import dicttoxml
import json
import glob
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

In [7]:
#data_split() fucntion will split data into 70-30 % and return train_set,test_set

def data_split():
    cwd=os.getcwd()
    data_dir=cwd+'\\Scrap'
    xml_loc_list=[]
    for file in os.listdir(data_dir):
        if file.endswith(".xml"):
            xml_loc=os.path.join(data_dir,file)
            xml_loc_list.append(xml_loc)
    random.shuffle(xml_loc_list)
    train ,test = train_test_split(xml_loc_list,test_size=0.25)
    data=[train,test]
    print("data_split")
    print("Train data:",len(train))
    print("Test data:",len(test))
    return data

In [8]:

'''
this fuction will check for corupt image and return a csv file.

use this fucntion on train, test set, to return train_csv nad test csv

csv contains img_location, img height, width, counding box co-ordinates,
class
'''
def data_to_csv(data):
    xml_list=[]
    for xml_loc in data:
        tree=ET.parse(xml_loc)
        root = tree.getroot()
        img_loc=xml_loc.split('.')
        img_loc.append('jpg')
        img_loc.remove('xml')
        img_loc='.'.join(img_loc)
        try:
            img=cv2.imread(img_loc)
        except:
            print("Image empyt: ",img_loc)
        for member in root.findall('object'):
            value=(img_loc,
                  member[0].text,
                   int(root.find('size')[0].text),
                   int(root.find('size')[1].text),
                   int(member[4][0].text),
                   int(member[4][1].text),
                   int(member[4][2].text),
                   int(member[4][3].text)
                  )
            xml_list.append(value)
            column_name=['filename','class','height','width',
                         'xmin','ymin','xmax','ymax']
            xml_df=pd.DataFrame(xml_list,columns=column_name)
    print(len(xml_df))
    return xml_df

In [9]:
#this is the ration on height,width with respect to 416 input size of yolo
def fx_fy(data_csv):
    fx = []
    fy = []
    for i in range(len(data_csv)):
        fx.append(416/int(data_csv['width'][i]))
        fy.append(416/int(data_csv['height'][i]))
    data_csv['fx']=fx
    data_csv['fy']=fy
    print(len(data_csv))
    return data_csv

In [10]:
def pre_processing():
    
    data=data_split()
    
    print("/n Notations")
    print("data_to_csv train_set")
    train_set=data_to_csv(data[0])
    
    print("data_to_csv test_set")
    test_set=data_to_csv(data[1])
    
    print('Train fx')
    train_csv=fx_fy(train_set)
    print('Train fy')
    test_csv=fx_fy(test_set)
    
    train_csv.to_csv(os.getcwd()+'\\data_csv\\train.csv')
    test_csv.to_csv(os.getcwd()+'\\data_csv\\test.csv')

In [13]:
pre_processing()

data_split
Train data: 279
Test data: 93
/n Notations
data_to_csv train_set
315
data_to_csv test_set
101
Train fx
315
Train fy
101
